Part 1:

Done:
-Use pretrained VGG
-Gather statistics from content image
-Gather statistics from style image

Part 2:

Done:
-Display style tensor and content tensor transformed back into image

Part 3:

Done:
-Create classes for style and content losses

Part 4:

Done:
-Used a BFGS optimizer
-Showed the content and style loss every 50 steps
-Clamp the outputs
-display the tensor as an image
![alt text](https://nolans-cs-bucket.s3-us-west-1.amazonaws.com/ezgif.com-gif-maker.gif)

In [0]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
import gc
from PIL import Image
import io
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose', color_scheme='LightBg', tb_offset = 1)

from google.colab import files

import pdb
from torch.utils import data

Part 1:



In [0]:
load_and_normalize = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((250,250)),
    transforms.ToTensor(),
])

def upload():
  print('Upload Content Image')
  file_dict = files.upload()
  content_path = io.BytesIO(file_dict[next(iter(file_dict))])

  print('\nUpload Style Image')
  file_dict = files.upload()
  style_path = io.BytesIO(file_dict[next(iter(file_dict))])
  return content_path, style_path

content_path, style_path = upload()

print("Content Path: {}".format(content_path))
print("Style Path: {}".format(style_path))

Upload Content Image


MessageError: ignored

In [0]:
content_image_orig = Image.open(content_path)
content_image = load_and_normalize(np.array(content_image_orig)).unsqueeze(0).cuda()
style_image_orig = Image.open(style_path)
style_image = load_and_normalize(np.array(style_image_orig)).unsqueeze(0).cuda()
dim=content_image.size(3)
import random


In [0]:
vgg_names = ["conv1_1", "relu1_1", "conv1_2", "relu1_2", "maxpool1", "conv2_1", "relu2_1", "conv2_2", "relu2_2", "maxpool2", "conv3_1", "relu3_1", "conv3_2", "relu3_2", "conv3_3", "relu3_3","maxpool3", "conv4_1", "relu4_1", "conv4_2", "relu4_2", "conv4_3", "relu4_3","maxpool4", "conv5_1", "relu5_1", "conv5_2", "relu5_2", "conv5_3", "relu5_3","maxpool5"]

vgg_names_style = ["conv2_1","conv3_1", "conv4_1", 
                  "conv5_1"]

style_idx=[]
for things in vgg_names_style:
  style_idx.append(vgg_names.index(things))

vgg_names_content = ["conv4_1"]

cont_idx=[]
for things in vgg_names_content:
  cont_idx.append(vgg_names.index(things))

# Choose the layers to use for style and content transfer

# Create the vgg network in eval mode
#  with our forward method that returns the outputs of the intermediate layers we requested



# Cache the outputs of the content and style layers for their respective images

In [0]:
import torchvision.models as models

class Normalization(nn.Module):
  def __init__(self, mean=torch.tensor([0.485, 0.456, 0.406]).cuda(), std=torch.tensor([0.229, 0.224, 0.225]).cuda()):
      super(Normalization, self).__init__()
      self.mean = torch.tensor(mean).view(-1, 1, 1)
      self.std = torch.tensor(std).view(-1, 1, 1)

  def forward(self, img):
      return (img - self.mean) / self.std

class VGGIntermediate(nn.Module):
  def __init__(self, requested=[]):
    super(VGGIntermediate, self).__init__()
    self.norm = Normalization().eval()
    self.intermediates = {}
    self.vgg = models.vgg16(pretrained=True).features.eval()
    for i, m in enumerate(self.vgg.children()):
        if isinstance(m, nn.ReLU):   # we want to set the relu layers to NOT do the relu in place. 
          m.inplace = False          # the model has a hard time going backwards on the in place functions. 
        
        if isinstance(m,nn.MaxPool2d):
          #pdb.set_trace()
          m=nn.AvgPool2d(2,stride=2)
        
        if i in requested:
          def curry(i):
            def hook(module, input, output):
              self.intermediates[i] = output
            return hook
          m.register_forward_hook(curry(i))
    
  def forward(self, x):
    self.vgg(self.norm(x))  
    return self.intermediates

Part 2

In [0]:
toPIL = transforms.ToPILImage()  

def display(tensor, title=None):
    image = tensor.cpu().clone()  
    image = image.squeeze(0)    
    image = toPIL(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)

plt.figure()
display(style_image, title='Style Image')

plt.figure()
display(content_image, title='Content Image')

Part 3

In [0]:
def gram_matrix(input):
  #pdb.set_trace()
  a,b,c,d = input.size()
  #a is the batch size
  #b is number f feature maps
  #c/d are dimensions of f map
  var = input.view(a*b,c*d)
  
  #test=np.random.shuffle(var)
  idx = torch.randperm(var.nelement())
  var = var.view(-1)[idx].view(var.size())
  var2 = torch.mm(var,var.t())
  
  out = var2.div(a*b*c*d)
    
  
  return out
   
  
class ContentLoss(nn.Module):
  def __init__(self, target):
    super(ContentLoss,self).__init__()
    self.target=target.detach()
       
  def forward(self,input,target_c):
    
    loss=torch.nn.functional.mse_loss(input[1],target_c[1])
    #pdb.set_trace()
    return loss#shouldn't this be loss?
    
class StyleLoss(nn.Module):
  def __init__(self,target_s):
    super(StyleLoss,self).__init__()
    
    self.target=gram_matrix(target_s).detach()
    
  def forward(self,input,target):
    G_input=gram_matrix(input[1])
    G_target=gram_matrix(target[1])
   
    loss=torch.nn.functional.mse_loss(G_input,G_target)
    return loss

content_layers=VGGIntermediate(cont_idx).cuda()

style_layers=VGGIntermediate(style_idx).cuda()

styleloss=StyleLoss(style_image)
contentloss=ContentLoss(content_image)
wl=0.20
#create white noise image with the same size


def get_losses(content_image,style_image,x,cont_idx,style_idx,content_layers,style_layers,wl):

  #get the single content layer
  P_out=list(content_layers(content_image).items())
  PX_out=list(content_layers(x).items())
  #get the multiple style layers
  S_out=list(style_layers(style_image).items())
  X_out=list(style_layers(x).items())

  loss_v=[]
  loss_c=[]
  #calculate the gram matrix for the style layers
  for i in range(len(S_out)):
   
    loss=styleloss(X_out[i],S_out[i])
    loss_v.append(loss*wl)
    #pdb.set_trace()
  loss_v_sum=sum(loss_v)
  
  for i in range(len(P_out)):
    #pdb.set_trace()
    loss=contentloss(PX_out[i],P_out[i])
    
    loss_c.append(loss)
  loss_c_sum=sum(loss_c)
    
  #calculate the loss for the style image
  
  
  #return the losses of each image
  return(loss_v_sum,loss_c_sum)

#white_image1=load_and_normalize(np.array([[random.random()*254 for i in range(dim)] for j in range(dim)],dtype='float32')).cuda()
#white_image2=load_and_normalize(np.array([[random.random()*254 for i in range(dim)] for j in range(dim)],dtype='float32')).cuda()
#white_image3=load_and_normalize(np.array([[random.random()*254 for i in range(dim)] for j in range(dim)],dtype='float32')).cuda()
content_copy=content_image



#test=(white_image.cpu().detach().numpy())




# Instantiate a content loss module for each content layer 
#  with the content reference image outputs for that layer for comparison

# Instantiate a sytle loss module for each style layer 
#  with the style reference image outputs for that layer for comparison

In [0]:
VGGIntermediate()

Part 4

In [0]:
epochs=100
#white_image1=load_and_normalize(np.array([[random.random()*254 for i in range(dim)] for j in range(dim)],dtype='float32')).cuda()
#white_image2=load_and_normalize(np.array([[random.random()*254 for i in range(dim)] for j in range(dim)],dtype='float32')).cuda()
#white_image3=load_and_normalize(np.array([[random.random()*254 for i in range(dim)] for j in range(dim)],dtype='float32')).cuda()

#white_image=torch.cat([white_image1,white_image2,white_image3]).unsqueeze(0)
#style_weight=1000
#content_weight=1000
content_copy=content_image

from google.colab import drive
def save_images(white_image,i):
  
  test=content_copy.squeeze(0)
  test2=test.cpu()
  test3=test2.detach().numpy()
  test3=test3
  r=test3[0]
  g=test3[1]
  b=test3[2]

  image=np.zeros((dim,dim,3))
  image[:,:,0]=r
  image[:,:,1]=g
  image[:,:,2]=b
  
  plt.imshow(image)

  drive.mount('/content/drive')
  with open('/content/drive/My Drive/stylized_image_' + str(i) + '.png', 'w') as f:
    out=plt.savefig('/content/drive/My Drive/stylized_image_' + str(i) + '.png')


def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.LBFGS([input_img.requires_grad_()])
    return optimizer
  
def style_transfer(content_image,style_image,content_copy,cont_idx,style_idx,content_layers,style_layers,wl,epochs,num_steps=50,
                       style_weight=5000, content_weight=1):
  optimizer=get_input_optimizer(content_copy)
  #pdb.set_trace()
 
  for i in range(epochs):
    
    def loop():
      
      content_copy.data.clamp_(0, 1)
      
      optimizer.zero_grad()

      loss_s,loss_c=get_losses(content_image,style_image,content_copy,cont_idx,style_idx,content_layers,style_layers,wl)

      loss_total=content_weight*loss_c+style_weight*loss_s
      #pdb.set_trace()
      loss_total.backward()
      print(loss_total)
      return(loss_total)
    optimizer.step(loop)
    content_copy.data.clamp_(0,1) 
    save_images(content_copy,i)
    print("images saved: " + str(i))
  
    
  
  
  
  
style_transfer(content_image,style_image,content_copy,cont_idx,style_idx,content_layers,style_layers,wl,epochs)

